In [1]:
%load_ext autoreload
%autoreload 2


In [9]:
!pip install --force-reinstall /home/boti/Desktop/ngram-trie/target/wheels/ngram_trie-0.1.0-cp312-cp312-linux_x86_64.whl

Processing /home/boti/Desktop/ngram-trie/target/wheels/ngram_trie-0.1.0-cp312-cp312-linux_x86_64.whl
  Attempting uninstall: ngram-trie
    Found existing installation: ngram_trie 0.1.0
    Uninstalling ngram_trie-0.1.0:
      Successfully uninstalled ngram_trie-0.1.0


In [11]:
import yaml
import torch
from utils.tokenizer import load_tokenizer
from train import model_from_config

device = "cuda" if torch.cuda.is_available() else "cpu"

tok = load_tokenizer("tokenizer_bytes")

with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    
with open("../configs/llama_medium.yaml", "r") as f:
    config = yaml.safe_load(f)
    


/tmp/ipykernel_2287/3938322137.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("../checkpoints/models/fit_check_bytes/23171.pt", map_location=de

<All keys matched successfully>

In [40]:
from typing import Iterator
from utils.sample import sample_with_temp, nucleus_sample
import torch

def generate_text(model, tokenizer, prompt, max_length=50, temperature=0.7)->Iterator[str]:
    model.eval()
    device = next(model.parameters()).device
    
    bos_token = tokenizer.token_to_id("<bos>")
    input_ids = torch.tensor([bos_token] + tokenizer.encode(prompt, add_special_tokens=False).ids).unsqueeze(0).to(device)
    
    generated = []
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(input_ids)
            next_token_logits = outputs.logits[:, -1, :]
            # next_token = nucleus_sample(next_token_logits, 0.5)
            next_token = sample_with_temp(next_token_logits, temperature)
        
        generated.append(next_token.item())
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=-1)
        
        # Check for end-of-sequence token
        if next_token.item() == tokenizer.token_to_id("<eos>"):
            break
    
        
        yield tokenizer.decode(next_token.tolist())

# Example usage
prompt = ""
print(prompt, end="")
for token in generate_text(model, tok, prompt, max_length=1024, temperature=0.5):
    print(token, end="")

 Lily was so happy that she hugged her mom and said, ""Thank you for helping me! You're the best!""" "Once upon a time, there was a little boy named Timmy. Timmy loved to play outside with his friends. One day, Timmy and his friends saw a big, scary tornado in the sky. They all ran inside to hide.  After the tornado passed, Timmy's friend Billy said, ""I'm scared. Can you help me?""  Timmy said, ""Don't worry, I'll protect you. We can hide in the closet until the tornado goes away.""  So, Timmy and Billy went to the closet and hid. They heard the tornado getting closer and closer. Suddenly, they heard a loud noise outside. They were so scared that they hid under the mattress.  After a while, the tornado went away. Timmy and his friends came out from under the mattress. ""Wow, that was scary!"" said Timmy. ""But we're safe now.""  From that day on, Timmy and his friends always played inside when there was a tornado. They knew they could always count on each other to keep them safe." "On

KeyboardInterrupt: 

In [69]:
with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    
# first 10000 words
data = " ".join(data.split()[:10000])
    
data = data.lower()
data = data.replace(".", "")
data = data.replace(",", "")
data = data.replace("!", "")
data = data.replace("?", "")
data = data.replace("'", "")
data = data.replace("\"", "")

word_to_ids = {word: i for i, word in enumerate(set(data.split()))}
ids_to_word = {i: word for word, i in word_to_ids.items()}

In [70]:
tokenized_data = [word_to_ids[word] for word in data.split()]

In [72]:
from ngram_trie import PyNGramTrie, PyModifiedBackoffKneserNey

# Create a new NGramTrie
trie = PyNGramTrie(n_gram_max_length=7, root_capacity=None)

trie.fit(tokenized_data, n_gram_max_length=7, root_capacity=None, max_tokens=None)

# Create a smoothing object
smoothing = PyModifiedBackoffKneserNey(trie)

# Get prediction probabilities
history = [word_to_ids[word] for word in "one day".split()]
probabilities = trie.get_prediction_probabilities(smoothing, history)
print(probabilities)

In [96]:
trie.search([word_to_ids[word] for word in "one day".split()]), data.count("one day")

(56, 56)

In [97]:
trie.search([word_to_ids[word] for word in "one day a".split()]), data.count("one day a")

(11, 11)

In [98]:
trie.search([word_to_ids[word] for word in "one day a".split()], "++*"), data.count("one day a")

(56, 11)

In [101]:
node = trie.find_all_nodes([word_to_ids[word] for word in "one day".split()])
[(ids_to_word[token], node.count) for token, node in node[0].children.items()]

[('a', 11),
 ('beep', 1),
 ('the', 7),
 ('while', 1),
 ('tim', 6),
 ('max', 1),
 ('when', 2),
 ('timmy', 1),
 ('his', 2),
 ('sue', 1),
 ('she', 8),
 ('he', 4),
 ('it', 1),
 ('zigzag', 1),
 ('they', 1),
 ('her', 1),
 ('tims', 1),
 ('joe', 1),
 ('mia', 1),
 ('lucy', 1),
 ('charlies', 1),
 ('sam', 1),
 ('bill', 1)]

In [106]:
data.count("one day while"), data.count("one day tim"), data.count("one day she")  # why is tim wrong?

(1, 8, 8)